In [5]:
import random
import operator
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.utils.extmath import density
from sklearn import metrics

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.linear_model import SGDClassifier,Perceptron,PassiveAggressiveClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import svm

### load some of the files

In [8]:
path_to_data = 'C:\\Users\\hbnp5049\\PycharmProjects\\untitled\\'# fill me!

In [5]:
training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)


In [6]:
email_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    for id in ids:
        email_sender[int(id)] = sender
#email_sender

In [7]:
es = pd.DataFrame.from_dict(email_sender, orient='index').reset_index()
es.rename(columns={'index': 'mid', 0: 'sender'}, inplace=True)
es.shape

(43613, 2)

In [8]:
m = pd.merge(training_info,es,on='mid',how='inner')

In [9]:
type(m)

pandas.core.frame.DataFrame

In [10]:
print(m.shape)
m.head(2)


(43613, 5)


,mid,date,body,recipients,sender
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...,christian.yoder@enron.com
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...,heather.dunton@enron.com


In [11]:
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids

# save all unique sender names (check it ! take off emails without @)
all_senders = emails_ids_per_sender.keys()

In [12]:
senders_dict = {value:index for index, value in enumerate(all_senders)}
#recipients_dict
'''
{'holli.krebs@enron.com': 0,
 'rmortensen@wn.com': 1,
 ...}
'''

"\n{'holli.krebs@enron.com': 0,\n 'rmortensen@wn.com': 1,\n ...}\n"

In [13]:
#senders_dict['holli.krebs@enron.com'] 

In [14]:
len(senders_dict)

125

In [15]:

'''
AttributeError: 'dict' object has no attribute 'iteritems'
In Python3 dict.items() does the same thing that dict.iteritems() did in Python2. 
Just replace iteritems() with items()!'''
# create address book with frequency information for each user
address_books = {}
i = 0

for sender, ids in emails_ids_per_sender.items():
    recs_temp = []
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        recs_temp.append(recipients)
    # flatten    
    recs_temp = [elt for sublist in recs_temp for elt in sublist]
    # compute recipient counts
    rec_occ = dict(Counter(recs_temp))
    # order by frequency
    sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    # save
    address_books[sender] = sorted_rec_occ
    
    #if i % 10 == 0:
    #    print(i)
    i += 1

'''Print inline - Change print item to:
print item, in Python 2.7
print(item, end=" ") in Python 3
If you want to print the data dynamically use following syntax:
print(item, sep=' ', end='', flush=True) in Python 3'''

# save all unique recipient names    
all_recs = list(set([elt[0] for sublist in address_books.values() for elt in sublist]))

# save all unique user names 
all_users = []
all_users.extend(all_senders)
all_users.extend(all_recs)
all_users = list(set(all_users))

In [16]:
recipients_dict = {value:index for index, value in enumerate(all_recs)}
#recipients_dict
'''
{'holli.krebs@enron.com': 0,
 'rmortensen@wn.com': 1,
 ...}
'''

"\n{'holli.krebs@enron.com': 0,\n 'rmortensen@wn.com': 1,\n ...}\n"

In [17]:
recipients_dict['clint.dean@enron.com']

1036

## Mappings senders

In [18]:
m.replace({"sender": senders_dict}, inplace=True)

In [19]:
m.head()

,mid,date,body,recipients,sender
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...,61
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...,70
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...,26
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com,74
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...,61


In [20]:
type(m['recipients'])

pandas.core.series.Series

In [21]:
type(m['recipients'][0])

str

In [22]:
m['recipients'][0].split(' ')

['robert.badeer@enron.com',
 'murray.o',
 'neil@enron.com',
 'tim.belden@enron.com',
 'elizabeth.sager@enron.com',
 'carla.hoffman@enron.com',
 'jeff.richter@enron.com',
 'chris.stokley@enron.com',
 'valarie.sabo@enron.com']

## mapping recipients

In [23]:
# this took 40 mn to run
for index, row in m.iterrows():
    recipients = row['recipients'].split() 
    recipients = [recipients_dict[rec] for rec in recipients if '@' in rec]
    m['recipients'][index] = recipients

C:\Users\OUMOUSS\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
mcols = m.columns

In [25]:
m.head(10)

,mid,date,body,recipients,sender
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,"[1670, 6450, 2590, 3349, 9745, 5882, 5461, 5555]",61
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,"[5983, 1670, 6450, 761, 6743, 2590, 3014, 4639...",70
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,"[1670, 9083, 2784]",26
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,"[1670, 5882]",74
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,"[4415, 3264, 1670, 6601]",61
5,105,2000-08-24 11:42:00,The new version of the EnronOnline website is ...,"[6408, 7294, 1511, 8763]",124
6,132,2000-08-27 23:11:00,Check this out. I think that we need to be si...,"[1670, 2795, 6911, 6743, 5882]",74
7,136,2000-08-28 02:50:00,We have had some confusion recently with respe...,"[1670, 6743, 2590, 3014, 7706, 7235, 5882, 174...",74
8,144,2000-08-28 08:15:00,I will be traveling to Calgary on Tuesday and ...,"[1670, 6450, 761, 6743, 2590, 3014, 356, 7706,...",74
9,167,2000-08-29 11:18:00,Please note that the EnronOnline Phase 2 train...,"[8276, 1455, 6808, 3014, 6044, 5701, 9083, 174...",124


In [ ]:
print(m['recipients'])
'''
0          [4631, 1509, 7562, 9149, 9118, 1442, 4454, 851]
'''
type(m['recipients'])

In [27]:
print(m['recipients'][0])
type(m['recipients'][0])

[1670, 6450, 2590, 3349, 9745, 5882, 5461, 5555]


list

In [28]:
m['recipients'][0][0]

1670

In [29]:
mainDF = m.copy()

## Saving the dataframe into a file 

In [30]:
### pay attention when you upload from file (recipients => str)

In [31]:
m.to_csv(r'C:\Users\OUMOUSS\PycharmProjects\DC17\DF6.csv', header=mcols, index=None, sep=',')

In [12]:
mainDF = pd.read_csv(path_to_data + 'DF6.csv', sep=',', header=0)
#mainDF.shape

In [13]:
# for testing 
mainDF = mainDF[:10000]

In [14]:
def convert_str_to_int(str):
    str = str[1:-1].split(', ')
    for index, s in enumerate(str) : 
        str[index] = int(s)
    return str

for index, row in mainDF.iterrows():
    recipients = convert_str_to_int(row['recipients'])
    mainDF['recipients'][index] = recipients

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
mainDF

In [34]:
from datetime import datetime
import datetime
from dateutil.parser import parse

In [35]:
# datetime.datetime.strptime(mainDF['date'], "%Y-%m-%d %H:%M:%S")
#pd.to_datetime(mainDF['date'])

In [36]:
mainDF['date'] = [parse(x) for x in mainDF['date']]

## Mail body preprocessing 

In [69]:
bodys = mainDF['body']

X_train = bodys[:900].values # numpy.ndarray
X_test = bodys[900:1000].values

In [53]:
# import nltk
# nltk.download()

In [70]:
# tokenizer=my_defined_tokenizer (callable)
vectorizer = TfidfVectorizer(sublinear_tf=True,
                             max_df=0.5,
                             stop_words='english',
                             analyzer = 'word',
                             max_features=150 )
X_train_bin = vectorizer.fit_transform(X_train)
X_test_bin = vectorizer.transform(X_test)

In [71]:
Y_train = mainDF['recipients'][:900].values
Y_test = mainDF['recipients'][900:1000].values

In [72]:
binarizer = MultiLabelBinarizer()
Y_train_bin = binarizer.fit_transform(Y_train)

In [59]:
Y_train_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
Y_train_bin.shape

(9000, 3866)

In [73]:
A = pd.DataFrame(X_train_bin.todense())
b = mainDF['sender'][:900].values
B = pd.DataFrame(data=b, columns=['sender'])
# c = mainDF['date'][:900].values
# C = pd.DataFrame(data=c, columns=['date'])
frame = [A, B]
df_train = pd.concat(frame, axis= 1)
df_train.shape

(900, 151)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity 

array([[ 1.        ,  0.        ,  0.19825746, ...,  0.0715866 ,
         0.04270442,  0.06600405]])

In [42]:
cosine_similarity(X_train_bin[0:1], X_train_bin).shape

(1, 9000)

In [45]:
df_train.shape

(9000, 151)

In [50]:
x = cosine_similarity(df_train[0:1], df_train)
x

array([[ 1.        ,  0.99991462,  0.9997845 , ...,  0.99988608,
         0.99988265,  0.99988541]])

In [51]:
x.argsort()

array([[1562, 8400, 8248, ...,  837, 3650,    0]], dtype=int64)

In [63]:
Y_train[0]

list

In [75]:
'''
KNeighborsClassifier(n_neighbors=10)
RandomForestClassifier(n_estimators=100)
OneVsRestClassifier(svm_clf)
OneVsRestClassifier(LinearSVC(class_weight='balanced'))
OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                          alpha=1e-4, n_iter=5, random_state=42,
                                          shuffle=True, n_jobs=-1))
classifier = OneVsRestClassifier(
SGDClassifier(
random_state = 0,
    loss ='hinge',
    alpha = 0.00001,
    penalty = 'lasticnet')).fit(features, labels)
'''

clf =OneVsRestClassifier(LinearSVC(class_weight='balanced'))
clf.fit(df_train, Y_train_bin)


OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [76]:
E = pd.DataFrame(X_test_bin.todense())
f = mainDF['sender'][900:1000].values
F = pd.DataFrame(data=f, columns=['sender'])
frame2 = [E, F]
df_test = pd.concat(frame2, axis= 1)

In [77]:
bin_pred = clf.predict(df_test)
bin_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [78]:
pred = binarizer.inverse_transform(bin_pred)
pred

[(37,
  54,
  70,
  107,
  115,
  121,
  130,
  195,
  245,
  267,
  279,
  285,
  332,
  362,
  380,
  386,
  441,
  481,
  537,
  538,
  613,
  615,
  653,
  726,
  742,
  744,
  748,
  937,
  955,
  958,
  1016,
  1024,
  1051,
  1054,
  1062,
  1111,
  1230,
  1295,
  1296,
  1315,
  1392,
  1415,
  1473,
  1501,
  1542,
  1562,
  1581,
  1587,
  1589,
  1651,
  1654,
  1759,
  1773,
  1825,
  1853,
  1867,
  2000,
  2043,
  2087,
  2089,
  2112,
  2130,
  2145,
  2223,
  2225,
  2253,
  2272,
  2296,
  2341,
  2347,
  2375,
  2399,
  2405,
  2520,
  2574,
  2668,
  2682,
  2728,
  2745,
  2833,
  2850,
  2914,
  2951,
  2969,
  2979,
  2985,
  3100,
  3108,
  3137,
  3146,
  3180,
  3195,
  3242,
  3262,
  3342,
  3358,
  3402,
  3405,
  3466,
  3507,
  3549,
  3576,
  3626,
  3645,
  3653,
  3655,
  3677,
  3685,
  3729,
  3822,
  3841,
  3929,
  3936,
  3977,
  4022,
  4042,
  4060,
  4074,
  4111,
  4122,
  4201,
  4205,
  4228,
  4243,
  4301,
  4360,
  4367,
  4405,
  4418,
 

In [79]:
len(pred[18]) # list

72

## Predictions 

In [77]:
print(type(B))
B

<class 'pandas.core.frame.DataFrame'>


,sender
0,61
1,70
2,26
3,74
4,61
5,124
6,74
7,74
8,74
9,124


In [91]:
type(mainDF['mid'][900:1000].values)

numpy.ndarray

In [92]:
type(mainDF['mid'][9000:10000])

pandas.core.series.Series

In [109]:
type(mids)

numpy.ndarray

In [80]:
mids = mainDF['mid'][9000:10000].values
df_mid = pd.DataFrame(data=mids, columns=['mid'])
df_mid

,mid
0,19064
1,19070
2,19073
3,19103
4,19105
5,19108
6,19116
7,19133
8,19135
9,19137


In [ ]:
pd.DataFrame(data=pred)

In [111]:
mid_recipients = {}
for idx, val in enumerate(mids):
    mid_recipients[val] = pred[idx]

In [112]:
mid_recipients

{19064: (1756,),
 19070: (1756, 8610),
 19073: (1756,),
 19103: (),
 19105: (1756,),
 19108: (1756,),
 19116: (1756, 4543),
 19133: (1756,),
 19135: (1756,),
 19137: (1756,),
 19138: (1756,),
 19139: (1756,),
 19147: (),
 19148: (1756,),
 19149: (1756,),
 19156: (1756,),
 19288: (),
 19289: (),
 19290: (),
 19291: (),
 19292: (),
 19293: (),
 19294: (),
 19295: (16,
  114,
  183,
  234,
  251,
  522,
  1235,
  2971,
  3542,
  4376,
  4452,
  4754,
  5052,
  7170,
  7443,
  7780,
  8765,
  8779,
  8859,
  9179,
  9354,
  9674),
 19296: (),
 19297: (),
 19298: (),
 19299: (),
 19300: (),
 19301: (),
 19302: (),
 19303: (),
 19304: (),
 19305: (),
 19310: (),
 19311: (),
 19312: (),
 19313: (),
 19314: (),
 19315: (),
 19316: (1534,),
 19317: (1534,),
 19318: (1534,),
 19319: (),
 19320: (),
 19321: (),
 19322: (),
 19323: (),
 19324: (),
 19325: (),
 19326: (),
 19327: (),
 19328: (),
 19329: (),
 19330: (),
 19331: (),
 19332: (),
 19333: (),
 19334: (),
 19335: (),
 19336: (),
 19337: 